# DATAKLASY

Idea - chcemy stworzyć klase reprezentującą karty, ich figury (ang. rank) i kolory (ang. suit) a następnie przy pomocy tej klasy móc tworzyć całe talie kart (ang. deck).

# Pomysł 1 - zwykła klasa

Klasa:

In [1]:
class RegularCard:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

Testowanie:

In [2]:
queen_of_hearts = RegularCard('Q', 'Hearts')
queen_of_hearts.suit, queen_of_hearts.rank

('Hearts', 'Q')

In [3]:
queen_of_hearts

In [4]:
queen_of_hearts == RegularCard('Q', 'Hearts')

False

Problematyczny tutaj jest fakt, ze nie wyświetla nam sie karta z jej kolorem i figurą a nazwa stworzonego obiektu co uniemożliwia przyrównywanie kart do siebie.

Sposobem na naprawe jest dodanie do klasy metod repr by je lepiej wyświetlać i eq by moc porównywać obiekty tej klasy.

Klasa:

In [5]:
class RegularCard:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        
    def __eq__(self, other): 
        if (self.rank==other.rank) and (self.suit==other.suit):
            return True
        else:
            return False  
        
    def __repr__(self):
        return f"RegularCard(rank = {self.rank}, suit = {self.suit})"

Testowanie:

In [6]:
queen_of_hearts = RegularCard('Q', 'Hearts')
queen_of_hearts.suit, queen_of_hearts.rank

('Hearts', 'Q')

In [7]:
queen_of_hearts

RegularCard(rank = Q, suit = Hearts)

In [8]:
queen_of_hearts == RegularCard('Q', 'Hearts')

True

Tak napisana klasa działa poprawnie, jednak wymaga to dosyć dużo kodu jak na tak prostą implementacje.

# Pomysł 2 - namedtuple

Drugim podejściem będzie skorzystanie z namedutple, czyli rozbudowanych tupli pozwalajacych tworzyć obiekty 

Namedtuple:

In [9]:
from collections import namedtuple

DataClassCard = namedtuple('DataClassCard', ['rank', 'suit'])

Testowanie:

In [10]:
queen_of_hearts = DataClassCard('Q', 'Hearts')
queen_of_hearts.rank, queen_of_hearts.suit

('Q', 'Hearts')

In [11]:
queen_of_hearts

DataClassCard(rank='Q', suit='Hearts')

In [12]:
queen_of_hearts == DataClassCard('Q', 'Hearts')

True

Namedtuple wydaje się działać w porządku. Jest to jednak złudne przeświadczenie, gdyż namedtuple jest podatne na sytuacje jak :

In [13]:
NotCard = namedtuple('NotCard', ['notrank', 'notsuit'])
queen_of_hearts == NotCard('Q', 'Hearts')

True

Tak więc namedtuple nie są dobrym rozwiązaniem, moge one jedynie być przydatne jako urozmaicenie metody repr, np gdy podobny 
schemat wyświetlania danych będzie sie powielał w kilku klasach:

In [14]:
DataClassCard = namedtuple('DataClassCard', ['rank', 'suit'])

class RegularCard:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        
    def __eq__(self, other): 
        if (self.rank==other.rank) and (self.suit==other.suit):
            return True
        else:
            return False  
        
    def __repr__(self):
        return str(DataClassCard(rank=self.rank, suit=self.suit))

In [15]:
queen_of_hearts = RegularCard('Q', 'Hearts')
queen_of_hearts.suit, queen_of_hearts.rank

('Hearts', 'Q')

In [16]:
queen_of_hearts

DataClassCard(rank='Q', suit='Hearts')

In [17]:
queen_of_hearts == RegularCard('Q', 'Hearts')

True

# Rozwiązanie - dataklasa

Najlepszym sposobem na stworzenie takiej klasy przy minimalnej dlugości kodu są dataklasy. Dataklasy to obiekty z pakietu o takiej samej nazwie będące uproszczeniem zwykłych klas poprzez automatyczne przypisanie podstawowych metod takich jak: 
- __init__ do definiowania zmiennych wewnątrz klasy
- __eq__ do porównywania obiektów tej samej klasy 
- __repr__ do wyświetlania obiektów danej klasy

bez koniecznosci ręcznego, osobistego definiowania, co oszczędza nam znaczną część kodu.

In [18]:
from dataclasses import dataclass

@dataclass
class DataClassCard:
    rank: str
    suit: str

Testowanie:

In [19]:
queen_of_hearts = DataClassCard('Q', 'Hearts')
queen_of_hearts.rank, queen_of_hearts.suit

('Q', 'Hearts')

In [20]:
queen_of_hearts

DataClassCard(rank='Q', suit='Hearts')

In [21]:
queen_of_hearts == DataClassCard('Q', 'Hearts')

True

Jak widać w powyższym przykładzie, w dataklasach stosuje się adnotacje typów do definiowania zmiennych. Tak jak w zwykłej klasie, możemy tutaj definiować różne typy zmiennych oraz przypisywać im wartości bazowe:

In [22]:
@dataclass
class aaa:
    rank: int = 1
    suit: str = "a"
    sit: float = 0.5   
aaa()

aaa(rank=1, suit='a', sit=0.5)

Mając już satysfakcjonującą klase naszych kart możemy przejść do drugiego etapu czyli tworzenia z nich talii. 

# Talie Kart 

Najbardziej podstawową implementacją talli kart będzie stworzenie listy złożonej z obiektów klasy kart, przy pomocy funkcji List[type].

In [23]:
from typing import List

@dataclass
class Deck:
    cards: List[DataClassCard]

In [24]:
queen_of_hearts = DataClassCard('Q', 'Hearts')
ace_of_spades = DataClassCard('A', 'Spades')
two_cards = Deck([queen_of_hearts, ace_of_spades])
two_cards

Deck(cards=[DataClassCard(rank='Q', suit='Hearts'), DataClassCard(rank='A', suit='Spades')])

Jest to działająca metoda, jednak jest ona prosta i kodogenna gdyż wymaga ona stworzenia 52 obiektów. By to usprawnić, wypiszemy dwie listy - dla figur i kolorów kart i stworzymy funkcję która stworzy nam ich wszystkie możliwe kombinacje:

In [25]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

RANKS, SUITS

(['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'],
 ['♣', '♢', '♡', '♠'])

In [26]:
def make_french_deck():
    return [DataClassCard(r, s) for s in SUITS for r in RANKS]
make_french_deck()

[DataClassCard(rank='2', suit='♣'),
 DataClassCard(rank='3', suit='♣'),
 DataClassCard(rank='4', suit='♣'),
 DataClassCard(rank='5', suit='♣'),
 DataClassCard(rank='6', suit='♣'),
 DataClassCard(rank='7', suit='♣'),
 DataClassCard(rank='8', suit='♣'),
 DataClassCard(rank='9', suit='♣'),
 DataClassCard(rank='10', suit='♣'),
 DataClassCard(rank='J', suit='♣'),
 DataClassCard(rank='Q', suit='♣'),
 DataClassCard(rank='K', suit='♣'),
 DataClassCard(rank='A', suit='♣'),
 DataClassCard(rank='2', suit='♢'),
 DataClassCard(rank='3', suit='♢'),
 DataClassCard(rank='4', suit='♢'),
 DataClassCard(rank='5', suit='♢'),
 DataClassCard(rank='6', suit='♢'),
 DataClassCard(rank='7', suit='♢'),
 DataClassCard(rank='8', suit='♢'),
 DataClassCard(rank='9', suit='♢'),
 DataClassCard(rank='10', suit='♢'),
 DataClassCard(rank='J', suit='♢'),
 DataClassCard(rank='Q', suit='♢'),
 DataClassCard(rank='K', suit='♢'),
 DataClassCard(rank='A', suit='♢'),
 DataClassCard(rank='2', suit='♡'),
 DataClassCard(rank='3', s

Zatem klase talli kart możemy więc stworzyć wywojując po prostu tą funkcje, prawda?

In [27]:
@dataclass
class Deck:
   cards: List[DataClassCard] = make_french_deck()

ValueError: mutable default <class 'list'> for field cards is not allowed: use default_factory

Otóż nie, bo wyskakuje error. Dzieje sie tak, gdyż tworzymy talię przy pomocy 'mutowalnego' obiektu. Oznacza to że zmiana tego obiektu (np usunięcie pojedynczej karty z pojedynczej talii) usunię ją również z każdej innej talii stworzonej przez tą metode.
Dlatego python jest wyuczony by odrzucać takie rozwiązania oraz jest specjalna komenda "field" stworzona by sobie radzić z mutowalnymi obiektami bazowymi poprzez indywidualne przypisywanie do nich wartości.

In [63]:
from dataclasses import field

@dataclass
class Deck:
    cards: List[DataClassCard] = field(default_factory=make_french_deck)

In [64]:
Deck()

Deck(cards=[DataClassCard(rank='2', suit='♣'), DataClassCard(rank='3', suit='♣'), DataClassCard(rank='4', suit='♣'), DataClassCard(rank='5', suit='♣'), DataClassCard(rank='6', suit='♣'), DataClassCard(rank='7', suit='♣'), DataClassCard(rank='8', suit='♣'), DataClassCard(rank='9', suit='♣'), DataClassCard(rank='10', suit='♣'), DataClassCard(rank='J', suit='♣'), DataClassCard(rank='Q', suit='♣'), DataClassCard(rank='K', suit='♣'), DataClassCard(rank='A', suit='♣'), DataClassCard(rank='2', suit='♢'), DataClassCard(rank='3', suit='♢'), DataClassCard(rank='4', suit='♢'), DataClassCard(rank='5', suit='♢'), DataClassCard(rank='6', suit='♢'), DataClassCard(rank='7', suit='♢'), DataClassCard(rank='8', suit='♢'), DataClassCard(rank='9', suit='♢'), DataClassCard(rank='10', suit='♢'), DataClassCard(rank='J', suit='♢'), DataClassCard(rank='Q', suit='♢'), DataClassCard(rank='K', suit='♢'), DataClassCard(rank='A', suit='♢'), DataClassCard(rank='2', suit='♡'), DataClassCard(rank='3', suit='♡'), DataCl

Jak widać nasz kod jest krótki a talia kompletna. Można tylko poprawić jej estetyke, poprzez ręczne zmodyfikowanie metod str i repr: 

In [85]:
@dataclass
class DataClassCard:
    rank: str
    suit: str
        
    def __str__(self):
        return f'{self.suit}{self.rank}'
    

@dataclass
class Deck:
    cards: List[DataClassCard] = field(default_factory=make_french_deck)

    def __repr__(self):
        cards = ', '.join(f'{c!s}' for c in self.cards)
        return f'{self.__class__.__name__}({cards})' 

In [86]:
Deck()

Deck(♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♣A, ♢2, ♢3, ♢4, ♢5, ♢6, ♢7, ♢8, ♢9, ♢10, ♢J, ♢Q, ♢K, ♢A, ♡2, ♡3, ♡4, ♡5, ♡6, ♡7, ♡8, ♡9, ♡10, ♡J, ♡Q, ♡K, ♡A, ♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♠A)

# Sortowanie

Naszym kolejnym a zarazem ostatnim celem bedzie możliwość porównywania mocy tych kart. Do tego skorzystamy z jednej z możlwiwych opcji dostępnej przy tworzeniu dataklasy - argumentu order:

In [32]:
@dataclass(order=True)

class DataClassCard:
    rank: str
    suit: str

In [33]:
queen_of_hearts = DataClassCard('Q', '♡')
ace_of_spades = DataClassCard('A', '♠')
ace_of_spades > queen_of_hearts

False

Bez określonej ręcznie metody porównywania kart, komenda order porównuje je jak obiekty którymi są, czyli tuple. Stąd ponieważ w alfabecie Q występuje później niż A to python uważa, że Q>A.

Zatem by móc sortowac i porównywać nasze karty, do każdej z nich dodamy liczbowy parametr sort_index, który będzie liczbowym wyznacznikiem ich mocy. Skorzystamy tutaj z faktu, że nasza wcześniejsza lista RANKS jest już posortowana, więc ich numer indeksu w liście będzie służył jako liczba mocy: 

In [68]:
@dataclass(order=True)
class DataClassCard:
    sort_index: int = field(init=False)
    rank: str
    suit: str

    def __post_init__(self):
        self.sort_index = (RANKS.index(self.rank))

    def __str__(self):
        return f'{self.suit}{self.rank}'

W tym kodzie należy spostrzec dwie rzeczy:

1. Parametr sort_index jest wyliczany w metodzie post_init.

2. Przy definiowaniu sort_index przypisujemy mu bazową wartość jako funkcje field gdzie init ma przypisaną wartość logiczną False. 

Obie te własności wynikają z faktu, że wartości sort_index są wyliczanie na podstawie innych parametrów podanych na początku dataklasy (tak jakby podanych w init). Stąd mamy specjalnie dopisany field, by go nie wliczać do "zawartości" inita oraz specjalną metode post init umożliwiającą wyliczanie wartości wybranych parametrów podanych w init na podstawie innych parametrów występujących w init.

Testowanie:

In [35]:
queen_of_hearts = DataClassCard('Q', '♡')
ace_of_spades = DataClassCard('A', '♠')
ace_of_spades > queen_of_hearts

True

In [36]:
ace_of_hearts = DataClassCard('A', '♡')
ace_of_spades <= ace_of_hearts

True

In [37]:
ace_of_hearts < queen_of_hearts 

False

Zatem możemy finalnie stworzyć posortowaną talię kart przy pomocy wbudowanej funkcji sorted:

In [38]:
Deck(sorted(make_french_deck()))

Deck(♠2, ♡2, ♢2, ♣2, ♠3, ♡3, ♢3, ♣3, ♠4, ♡4, ♢4, ♣4, ♠5, ♡5, ♢5, ♣5, ♠6, ♡6, ♢6, ♣6, ♠7, ♡7, ♢7, ♣7, ♠8, ♡8, ♢8, ♣8, ♠9, ♡9, ♢9, ♣9, ♠10, ♡10, ♢10, ♣10, ♠J, ♡J, ♢J, ♣J, ♠Q, ♡Q, ♢Q, ♣Q, ♠K, ♡K, ♢K, ♣K, ♠A, ♡A, ♢A, ♣A)

Finalne pytanie może brzmieć czy dataklasa przy definiowaniu może zawierać jeszcze jakieś inne parametry oprócz tutaj wymienionych. Istnieją jeszcze dwie metody: frozen i unsafe_hash, które domyślnie przybierają wartości False, i są powiązane z ustawieniami co do mutowalności tworzonej dataklasy (zmiana ich wartości na True powoduje niemutowalność dataklasy, czyli nie możemy do niej przypisać własnej wartości).

In [39]:
@dataclass(init=True, eq=True, repr=True, order=True, frozen=True, unsafe_hash=True)
class aaa:
    rank: int = 1
    suit: str = "a"
    sit: float = 0.5   
aaa()

aaa(rank=1, suit='a', sit=0.5)